In [ ]:
# 大学教育の付加価値検証：学習習慣・アルバイトとGPAの構造分析
### Value-Added in Higher Education: Factors of GPA, Study Habits, and Part-time Work

![Python](https://img.shields.io/badge/Python-3.10+-blue.svg)
![IR](https://img.shields.io/badge/Focus-Student%20Success%20%2F%20Learning%20Analytics-green.svg)

---

## 🇯🇵 日本語概要 (Executive Summary)

本プロジェクトは、大学教育における最重要指標である **GPA (Grade Point Average)** の決定要因を、「入学時学力（Input）」だけでなく「生活習慣・学習方略・アルバイト（Environment）」の観点から多次元的に分析したものです。

分析の結果、GPAは単なる学習時間の長短ではなく、**「経済的制約による時間の貧困」** と **「学習方略（質）の巧拙」** の相互作用によって決定される構造を明らかにしました。

👉 **[詳細レポート全文を読む (PDF)](docs/Value_Added_in_Higher_Education.pdf)**

---

## 📘 Project Overview

This study analyzes the determinants of GPA using the **I-E-O (Input-Environment-Output) model**. It highlights that simply increasing "study time" is not enough to improve academic performance. Instead, **"Learning Strategies"** and **"Economic Constraints (Time Poverty)"** are the critical factors.

### 📊 Key Findings (Simulation)

1.  **The Limits of Input:** Entrance exam scores correlate with 1st-year GPA but fade out in later years.
2.  **Time Poverty:** Working part-time for **more than 20 hours/week** significantly lowers GPA.
3.  **Quality over Quantity:** "Deep Learning Strategies" have a higher impact on GPA than mere study hours.

---

## 🛠 Methodology

* **Data Generation:** Simulating student data (N=3500) based on Japanese higher education trends (JASSO surveys).
* **Analysis:** OLS Regression to identify the weight of each factor (Sleep, Work, Strategy) on GPA.
* **Tools:** Python (`pandas`, `statsmodels`, `numpy`)

### 📂 Repository Structure
* `value_added_analysis.ipynb`: Python code for data generation and regression analysis.
* `docs/`: Full report in PDF format.

---

## 👤 Author
**Keisuke Nakamura (keisuke-data-lab)**
*Data Analyst / University IR Specialist*

---
> © 2025 Keisuke Nakamura